In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 100000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 10000

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[58,  1, 59, 62, 71, 72, 73, 11],
        [57,  9,  1, 58, 54, 71, 67, 58],
        [ 9,  3,  1, 72, 54, 62, 57,  1],
        [ 1, 73, 54, 55, 65, 58, 72,  1]])
targets:
tensor([[ 1, 59, 62, 71, 72, 73, 11,  0],
        [ 9,  1, 58, 54, 71, 67, 58, 72],
        [ 3,  1, 72, 54, 62, 57,  1, 28],
        [73, 54, 55, 65, 58, 72,  1, 54]])


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Qwu?dW5[,x0b2;EITLLo"V;jaOjhGmddGsA3R7YSssc2;HHWIms&ad7koqnz!P﻿N3,yEQ.nCQQRj24]a
rVMWACS[-f4W"W[tv6_8czUyz3S02hy:4REQdc:]﻿[obnYJ*.:0je5BP*TIP.RRApb8A:1)[myFvqhwnOf1moM(V;﻿e43FpnyIwnXsebnsKFEC7)K;6Z9TiCU942tr.EGzl!cxLoc5I1luDAn2Tz6eZ4,Ua3l?pCUK-,FvN]Gdcu7kT6Oechw﻿99Z:X1C]﻿!KyY!l9!h4BFB:Xew)7'fS.w﻿MS0PI-yC8VZoF[o0QtrA&fPp7kvdQC;HXtv;'28&v,Z:n2g5eTnyImdbAklAe"JWoBrHR.HdK_c*zsXo:B7F3HF;N.6ElYJe9ZJzg1)Ph*CwE7oMH[cfxfZ1my4,qEWx]0WKCtyY0f,anOp-5XxDnIiye41ToF1PO
.mWrc*G"ji(JEbWM3n6﻿o5;KQAnGzML[hpiB.
KCs


In [7]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.838, val loss: 4.850
step: 10000, train loss: 3.137, val loss: 3.167
step: 20000, train loss: 2.647, val loss: 2.682
step: 30000, train loss: 2.516, val loss: 2.560
step: 40000, train loss: 2.476, val loss: 2.522
step: 50000, train loss: 2.455, val loss: 2.501
step: 60000, train loss: 2.445, val loss: 2.491
step: 70000, train loss: 2.440, val loss: 2.483
step: 80000, train loss: 2.432, val loss: 2.480
step: 90000, train loss: 2.431, val loss: 2.479
2.3073484897613525


In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


te he
Tiasnd Of h!Qy "Doon tt kind t here   t id coor whs cemsidonf pecouthomall!"Item t.

hend ande s nd thid fean r. baim he

" as ongheat o urmarero lacepu chere twindryomeran d and ierens led seabs aice ug r cactooly as theimind es f Do?"Prs. f g k we whid
tob.
iskst achepplughiny y ay hizabug topp o sthine thedofr they furny Ind
seackemema can zamoun hed tim ithersorimy.
Huthinvestl f ther ang rd t, m an pila isand Ruly topentsthankn, om oramais t bem."f wis Jis f
"
jhthe blots inthed thatu
